In [1]:
#Box number 1
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys

In [2]:
#Box number 2
#Upload datasets into dataframes
off_week_data_raw = pd.DataFrame(pd.read_csv("weekly_team_stats_offense.csv")) #Read the data file for offense and defense by week  
def_week_data_raw = pd.DataFrame(pd.read_csv("weekly_team_stats_defense.csv"))

#Clean the file to only include REG season and 2024 (this is where you go to change the year)
def clean_week_data(df): 
    df = df[df['season_type'] == 'REG']
    df = df[df['season'] == 2024]
    return df

#Uses the function to clean the data
off_week_data = clean_week_data(off_week_data_raw)
def_week_data = clean_week_data(def_week_data_raw)

In [15]:
#Box number 3
#Function to get the previous weeks data depending on the week, team and dataframe entered
def get_previous_weeks_home(df, home_team, week):
    off_home_team = df[df['team'] == home_team]
    if week == 1: #Check to make sure we have enough evidence to make the calculation
        print("Can not calculate from no data please make sure the week is 2 or above.")
        sys.exit() #Force end the program if week is 1
        
    if week > 18: #Check to make sure we don't go to weeks greater than 18
        print("Can not go into the postseason there are only 18 weeks in the regular season.")
        sys.exit() #Force end the program if week is higher than 18
        
    return off_home_team[off_home_team["week"] < week] #Gives the data from the previous weeks before week into the dataframe

In [21]:
#Box number 4
#Function to do the same as the last one, but it finds the away team based on the home team selected
def get_previous_weeks_away(df, home_team, week):
    if week == 1: #Check to make sure we have enough evidence to make the calculation
        print("Can not calculate from no data please make sure the week is 2 or above.")
        sys.exit() #Force end the program if week is 1
        
    if week > 18: #Check to make sure we don't go to weeks greater than 18
        print("Can not go into the postseason there are only 18 weeks in the regular season.")
        sys.exit() #Force end the program if week is higher than 18
    off_home_team = df[df['team'] == home_team]
    game = off_home_team[off_home_team['week'] == week]['game_id'].values
    df2 = df[df['game_id'].isin(game)] #Gives the away team using the home team 
    away_team = df2[df2['team'] != home_team]['team'].values[0]
    away_team_df = df[(df['team'] == away_team)&(df['week']<week)] #Makes the dataframe using all the data from the away team
    return away_team_df, away_team #Returns the dataframe of prior weeks and away team name

    

In [17]:
#Box number 5
def home_points(df_off, df_def, home_team, week):
    total_points = 0 
    #Makes dataframes for the offence and defence of the team selected
    off_home_team = off_week_data[off_week_data['team'] == home_team]
    def_home_team = def_week_data[def_week_data['team'] == home_team]
    previous_offense_home = get_previous_weeks_home(df_off, home_team, week)
    previous_defense_home = get_previous_weeks_home(df_def, home_team, week)
    #Point calculations based on all of the variables selected
    total_points = total_points + previous_offense_home['total_off_yards'].sum() 
    total_points = total_points + (previous_offense_home['total_tds'].sum() * 70)
    total_points = total_points + (previous_offense_home['field_goal'].sum() * 30)
    total_points = total_points + (previous_offense_home['extra_point'].sum() * 10)
    total_points = total_points + ((previous_offense_home['first_down_rush'].sum() + previous_offense_home['first_down_pass'].sum()) * 10)
    total_points = total_points + (previous_defense_home['sack'].sum() * 20)
    total_points = total_points + (previous_defense_home['interception'].sum() * 50)
    return total_points, home_team #Returns the point total of prior weeks and home team name

In [18]:
#Box number 6
def away_points(df_off, df_def, home_team, week):
    total_points = 0
    #Makes dataframes for the offence and defence of the team selected
    previous_offense_home = get_previous_weeks_away(df_off, home_team, week)[0]
    previous_defense_home = get_previous_weeks_away(df_def, home_team, week)[0]
    away_team = get_previous_weeks_away(df_def, home_team, week)[1] #Get the away teams name
    off_away_team = off_week_data[off_week_data['team'] == away_team]
    def_away_team = def_week_data[def_week_data['team'] == away_team]
    #Point calculations based on all of the variables selected, dataset information, and point calculation
    total_points = total_points + previous_offense_home['total_off_yards'].sum()
    total_points = total_points + (previous_offense_home['total_tds'].sum() * 70)
    total_points = total_points + (previous_offense_home['field_goal'].sum() * 30)
    total_points = total_points + (previous_offense_home['extra_point'].sum() * 10)
    total_points = total_points + ((previous_offense_home['first_down_rush'].sum() + previous_offense_home['first_down_pass'].sum()) * 10)
    total_points = total_points + (previous_defense_home['sack'].sum() * 20)
    total_points = total_points + (previous_defense_home['interception'].sum() * 50)
    return total_points, away_team #Returns the point total of prior weeks and away team name

In [19]:
#Box number 7
def win_percentage(home, away):
    #Average it out and then get the percentage of each
    total = home + away 
    home_percentage = home/total*100
    away_percentage = away/total*100

    return home_percentage, away_percentage 

In [20]:
#Box number 8
#Allows the user to give an input for both the team and week
team = str(input("Enter a team: "))
week = int(input("Enter what week you want calculated: "))

#Excepts an index error as a bye week to prevent confusion when running the program
try:
    h_point_total = home_points(off_week_data, def_week_data, team, week)[0]
    a_point_total = away_points(off_week_data, def_week_data, team, week)[0]
except IndexError as e:
    print("That team is on a bye this week, please try again.")
    sys.exit()

#Assigns the home team and the away team to variables
home_team = home_points(off_week_data, def_week_data, team, week)[1] 
away_team = away_points(off_week_data, def_week_data, team, week)[1]
home_pct, away_pct = win_percentage(h_point_total, a_point_total)

#Gives the percentage for both teams for 2 decimals points
print(f"{home_team}: {home_pct:.2f}% | {away_team}: {away_pct:.2f}%") 

#Sets the labels and sizes varibales for the pie chart
labels = [f'{home_team} Team Wins', f'{away_team} Team Wins']
sizes = [home_pct, away_pct]

#Give a figure size
plt.figure(figsize=(6,6)) 

plt.pie(sizes, labels=labels, autopct='%1.2f%%', startangle=90, colors = ['deepskyblue', 'orangered']) #Gives the Numbers for 2 decimal points and starts at the top
plt.title(f'{home_team} vs {away_team} Win Percentage')
plt.axis('equal')  # Makes the chart a pie

plt.show()

Enter a team:  GB
Enter what week you want calculated:  1


Can not calculate from no data please make sure the week is 2 or above.


SystemExit: 